In [ ]:
!nvidia-smi

Fri Nov 11 17:53:23 2022       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 460.32.03    Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla T4            Off  | 00000000:00:04.0 Off |                    0 |
| N/A   38C    P8     9W /  70W |      0MiB / 15109MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [ ]:
!pip install transformers
!pip install datasets
!pip install sacrebleu

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 5.5 MB 15.8 MB/s 
     |████████████████████████████████| 7.6 MB 56.6 MB/s 
     |████████████████████████████████| 163 kB 67.0 MB/s 
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 441 kB 14.8 MB/s 
     |████████████████████████████████| 95 kB 5.7 MB/s 
     |████████████████████████████████| 115 kB 62.5 MB/s 
     |████████████████████████████████| 212 kB 74.1 MB/s 
     |████████████████████████████████| 127 kB 67.1 MB/s 
     |████████████████████████████████| 115 kB 69.7 MB/s 
  Attempting uninstall: urllib3
    Found existing installation: urllib3 1.24.3
    Uninstalling urllib3-1.24.3:
      Successfully uninstalled urllib3-1.24.3
  Attempting uninstall: dill
    Found existing installation: dill 0.3.6
    Uninstalling dill-0.3.6:
      Successfully un

In [ ]:
from transformers import AutoTokenizer
from datasets import load_dataset, load_metric

In [ ]:
# raw_datasets = load_dataset("wmt16", "de-en")
raw_datasets = load_dataset("wmt16", "de-en")
metric = load_metric("sacrebleu")

Extracting data files:   0%|          | 0/5 [00:00<?, ?it/s]

Extracting data files: 0it [00:00, ?it/s]

Generating train split:   0%|          | 0/4548885 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/2169 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/2999 [00:00<?, ? examples/s]

Dataset wmt16 downloaded and prepared to /root/.cache/huggingface/datasets/wmt16/de-en/1.0.0/746749a11d25c02058042da7502d973ff410e73457f3d305fc1177dc0e8c4227. Subsequent calls will reuse this data.


  0%|          | 0/3 [00:00<?, ?it/s]

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:3: FutureWarning: load_metric is deprecated and will be removed in the next major version of datasets. Use 'evaluate.load' instead, from the new library 🤗 Evaluate: https://huggingface.co/docs/evaluate
  This is separate from the ipykernel package so we can avoid doing imports until


In [ ]:
print(raw_datasets)

DatasetDict({
    train: Dataset({
        features: ['translation'],
        num_rows: 4548885
    })
    validation: Dataset({
        features: ['translation'],
        num_rows: 2169
    })
    test: Dataset({
        features: ['translation'],
        num_rows: 2999
    })
})


In [ ]:
raw_dataset_train = load_dataset("wmt16", "de-en", split="train[:20%]")
raw_dataset_test = load_dataset("wmt16", "de-en", split="test[:20%]")
raw_dataset_validation = load_dataset("wmt16", "de-en", split="validation[:20%]")

In [ ]:
print(raw_dataset_train)
print(raw_dataset_test)
print(raw_dataset_validation)

Dataset({
    features: ['translation'],
    num_rows: 909777
})
Dataset({
    features: ['translation'],
    num_rows: 600
})
Dataset({
    features: ['translation'],
    num_rows: 434
})


In [ ]:
from datasets import DatasetDict 
raw_datasets = DatasetDict({'train': raw_dataset_train, 'test': raw_dataset_test, 'validation': raw_dataset_validation})

In [ ]:
print(raw_datasets)

DatasetDict({
    train: Dataset({
        features: ['translation'],
        num_rows: 909777
    })
    test: Dataset({
        features: ['translation'],
        num_rows: 600
    })
    validation: Dataset({
        features: ['translation'],
        num_rows: 434
    })
})


In [ ]:
predictions = ["hello there general kenobi", "foo bar foobar"]
references = [["hello there general kenobi", "hello there !"],["foo bar foobar", "foo bar foobar"]]
results = metric.compute(predictions=predictions, references=references)
print(round(results["score"], 1))

100.0


In [ ]:
tokenizer = AutoTokenizer.from_pretrained("t5-small")

Downloading:   0%|          | 0.00/1.20k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/792k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/1.39M [00:00<?, ?B/s]

/usr/local/lib/python3.7/dist-packages/transformers/models/t5/tokenization_t5_fast.py:166: FutureWarning: This tokenizer was incorrectly instantiated with a model max length of 512 which will be corrected in Transformers v5.
For now, this behavior is kept to avoid breaking backwards compatibility when padding/encoding with `truncation is True`.
- Be aware that you SHOULD NOT rely on t5-small automatically truncating your input to 512 when padding/encoding.
- If you want to encode/pad to sequences longer than 512 you can either instantiate this tokenizer with `model_max_length` or pass `max_length` when encoding/padding.
- To avoid this warning, please instantiate this tokenizer with `model_max_length` set to your preferred value.
  FutureWarning,


In [ ]:
prefix = "translate English to German: "

In [ ]:
max_input_length = 128
max_target_length = 128
source_lang = "en"
target_lang = "de"

def preprocess_function(examples):
    inputs = [prefix + ex[source_lang] for ex in examples["translation"]]
    targets = [ex[target_lang] for ex in examples["translation"]]
    model_inputs = tokenizer(inputs, max_length=max_input_length, truncation=True, padding=True)

    # Setup the tokenizer for targets
    with tokenizer.as_target_tokenizer():
        labels = tokenizer(targets, max_length=max_target_length, truncation=True, padding=True)

    model_inputs["labels"] = labels["input_ids"]
    return model_inputs

In [ ]:
# from datasets import load_from_disk
# tokenized_datasets = load_from_disk('./tokenizer')

In [ ]:
tokenized_datasets = raw_datasets.map(preprocess_function, batched=True)

  0%|          | 0/910 [00:00<?, ?ba/s]

/usr/local/lib/python3.7/dist-packages/transformers/tokenization_utils_base.py:3547: UserWarning: `as_target_tokenizer` is deprecated and will be removed in v5 of Transformers. You can tokenize your labels by using the argument `text_target` of the regular `__call__` method (either in the same call as your input texts if you use the same keyword arguments, or in a separate call.
  "`as_target_tokenizer` is deprecated and will be removed in v5 of Transformers. You can tokenize your "


  0%|          | 0/1 [00:00<?, ?ba/s]

  0%|          | 0/1 [00:00<?, ?ba/s]

In [ ]:
tokenized_datasets.save_to_disk('./tokenizer')

In [ ]:
print(tokenized_datasets['train'][0])

{'translation': {'de': 'Wiederaufnahme der Sitzungsperiode', 'en': 'Resumption of the session'}, 'input_ids': [13959, 1566, 12, 2968, 10, 419, 4078, 102, 1575, 13, 8, 2363, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], 'attention_mask': [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], 'labels': [15158, 24860, 74, 11216, 425, 7, 4267, 32, 221, 1, 0, 0, 0, 0, 

In [ ]:
print(tokenizer.decode(tokenized_datasets['train'][0]['input_ids'], skip_special_tokens=True))

translate English to German: Resumption of the session


In [ ]:
from transformers import AutoModelForSeq2SeqLM

model = AutoModelForSeq2SeqLM.from_pretrained("t5-small")

Downloading:   0%|          | 0.00/242M [00:00<?, ?B/s]

In [ ]:
from transformers import DataCollatorForSeq2Seq

data_collator = DataCollatorForSeq2Seq(tokenizer=tokenizer, model=model)

In [ ]:
from tensorflow.python.ops.math_ops import mean
from transformers import Seq2SeqTrainingArguments, Seq2SeqTrainer
from torch import nn
from torch.distributions import Categorical
import sys, torch

class CustomTrainer(Seq2SeqTrainer):
    def __init__(self, *args, **kwargs):
        super().__init__(*args, **kwargs)
        self.bleu = []
        self.distributions = []
        self.counter = 0 

    def compute_loss(self, model, inputs, return_outputs=False,alpha=float):
        log_probs = []
        labels = inputs.get("labels")
        print(labels.shape)
        # forward pass
        outputs = model(**inputs)
        logits = outputs.get("logits")
        print(len(logits))

        attn_mask = torch.where(labels != -100, inputs['attention_mask'], 0)
        masked_labels = labels*attn_mask
        print(masked_labels.shape)
        # decoded_labels = []
        # for i in range(len(masked_labels)):
        #   decoded_labels.append([tokenizer.decode(masked_labels[i], skip_special_tokens=True)])
        decoded_labels = tokenizer.batch_decode(masked_labels,skip_special_tokens=True)
        decoded_labels = [[i] for i in decoded_labels]

        attn_mask = torch.unsqueeze(attn_mask, 2)
        masked_logits = logits*attn_mask
        print(masked_logits.shape)
        decoded_logits = []
        log_probs_batch = []
        # for i in range(len(masked_logits)):
        #   try:
        #     pad_idx = ((masked_logits[i] == 0).nonzero(as_tuple=True)[0])[0].item()
        #   except:
        #     pad_idx = len(masked_logits[i])
        #   log_probs = 0
        #   word_list = []
        #   for j in range(0, pad_idx):
        #     cur_logit = masked_logits[i][j]
        #     distrib = Categorical(logits=cur_logit)
        #     next_word = distrib.sample()
        #     log_probs += distrib.log_prob(next_word)
        #     word_list.append(next_word)
        #   log_probs /= pad_idx
        #   log_probs_batch.append(log_probs)
        #   decoded_logits.append(tokenizer.decode(word_list, skip_special_tokens=True))
        # print(decoded_logits)
        # print(log_probs_batch)
        cat = Categorical(logits=masked_logits)
        samplebatch = cat.sample()
        log_probs_batch = cat.log_prob(samplebatch)
        log_probs_batch = torch.mean(log_probs_batch,1)
        decoded_logits = tokenizer.batch_decode(samplebatch, skip_special_tokens=True)
        print(len(decoded_labels))
        print(len(decoded_logits))
        print(log_probs_batch.shape)

        bleu_scores = [round(metric.compute(predictions=decoded_logits, references=decoded_labels)["score"], 1)]
        # self.bleu.append(sum(bleu_scores))
        # self.counter += len(bleu_scores)
        # avg_bleu = sum(self.bleu)/self.counter
        # bleu_scores = [score - avg_bleu for score in bleu_scores]
        
        # compute custom loss (suppose one has 3 labels with different weights)
        # probs = []
        loss_ce = [nn.CrossEntropyLoss(masked_logits[i],masked_labels[i]) for i in range(len(masked_logits))]
        loss_re = sum([-log_prob*bleu_score for log_prob, bleu_score in zip(log_probs_batch, bleu_scores)])
        #loss = (alpha*loss_ce) + ((1-alpha)*loss_re)

        # for i in range(len(logits)):
        #   ce_loss = loss_fct(logits[i], labels[i])
        #   probs.append(ce_loss)
        sys.exit(1)
        # loss = loss_fct(logits.view(-1, logits.size(-1)), labels.view(-1))
        # print(loss)
        #return (loss, outputs) if return_outputs else loss

In [ ]:
from transformers import Seq2SeqTrainingArguments

training_args = Seq2SeqTrainingArguments(
    output_dir="./results",
    evaluation_strategy="epoch",
    learning_rate=2e-5,
    per_device_train_batch_size=16,
    per_device_eval_batch_size=16,
    weight_decay=0.01,
    save_total_limit=3,
    num_train_epochs=1,
    fp16=True,
)

trainer = CustomTrainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_datasets["train"],
    eval_dataset=tokenized_datasets["test"],
    tokenizer=tokenizer,
    data_collator=data_collator,
)

PyTorch: setting up devices
The default value for the training argument `--report_to` will change in v5 (from all installed integrations to none). In v5, you will need to use `--report_to all` to get the same behavior as now. You should start updating your code and make this info disappear :-).
Using cuda_amp half precision backend


In [ ]:
trainer.train()

In [ ]:
for data in tqdm(tokenized_datasets["validation"]):
  label = data["translation"]["de"]
  output = model.generate(torch.LongTensor(data["input_ids"]).cuda().unsqueeze(0))
  tr = tokenizer.decode(output[0],skip_special_tokens=True)
  print(data["translation"]['en'])
  print(label)
  print(tr)
  break